# Desafio Kinea


## Importando

In [1]:
import logging
logging.level=logging.INFO

In [2]:
from typing import Dict, List
from datetime import datetime

In [3]:
from ambima_connect import AmbimaConnect
import utils
from titulos import LTN, NTNB, NTNF, LFT

## Interface com API Ambima

In [4]:
data_list = AmbimaConnect('titulos publicos').content

In [5]:
priv = AmbimaConnect('titulos privados').content

In [6]:
priv

[{'grupo': 'DI PERCENTUAL',
  'codigo_ativo': 'AESL14',
  'data_referencia': '2017-10-20',
  'data_vencimento': '2020-10-20',
  'percentual_taxa': '114,5% do DI',
  'taxa_compra': 110.75,
  'taxa_venda': 108.25,
  'taxa_indicativa': 109.4821,
  'desvio_padrao': 0.03,
  'val_min_intervalo': 109.45,
  'val_max_intervalo': 109.52,
  'pu': 10092.552541,
  'percent_pu_par': 100.93,
  'duration': 575,
  'percent_reune': '--',
  'emissor': 'AES SUL DISTRIBUIDORA GAUCHA DE ENERGIA S/A (*) (**)'},
 {'grupo': 'DI SPREAD',
  'codigo_ativo': 'AGGU12',
  'data_referencia': '2017-10-20',
  'data_vencimento': '2017-11-23',
  'percentual_taxa': 'DI + 1,40%',
  'taxa_compra': 1.0325,
  'taxa_venda': 0.9,
  'taxa_indicativa': 0.9723,
  'desvio_padrao': 0.02,
  'val_min_intervalo': 0.95,
  'val_max_intervalo': 0.99,
  'pu': 6819.835545,
  'percent_pu_par': 100.52,
  'duration': 311,
  'percent_reune': '--',
  'emissor': 'ÁGUAS GUARIROBA S/A (*) (**)'},
 {'grupo': 'IPCA SPREAD',
  'codigo_ativo': 'AGRU11'

In [7]:
def clean_list(list_to_clean:List[Dict]):
  keep_keys = {'tipo_titulo','data_referencia','data_vencimento','taxa_indicativa','pu'}
  keep_keys.update(['grupo', 'codigo_ativo', 'percentual_taxa', 'duration', 'referencia_ntnb'])
  res = [{key : val for key, val in sub.items() if key in keep_keys} for sub in list_to_clean]
  return res

In [8]:
clean_priv = clean_list(priv)

In [9]:
clean_mercado = clean_list(data_list)

In [10]:
clean_priv[0]

{'grupo': 'DI PERCENTUAL',
 'codigo_ativo': 'AESL14',
 'data_referencia': '2017-10-20',
 'data_vencimento': '2020-10-20',
 'percentual_taxa': '114,5% do DI',
 'taxa_indicativa': 109.4821,
 'pu': 10092.552541,
 'duration': 575}

In [11]:
[x['estimativa_taxa_selic'] for x in AmbimaConnect('selic').content if x['data_referencia'] == '2017-10-20'][0]

8.15

In [12]:
from datetime import date
'2020-02-02' == date(2020,2,2)

False

In [13]:
clean_priv[0]

{'grupo': 'DI PERCENTUAL',
 'codigo_ativo': 'AESL14',
 'data_referencia': '2017-10-20',
 'data_vencimento': '2020-10-20',
 'percentual_taxa': '114,5% do DI',
 'taxa_indicativa': 109.4821,
 'pu': 10092.552541,
 'duration': 575}

## Cálculo Geral

In [16]:
for titulo in clean_mercado:

  if 'tipo_titulo' in titulo:
    if titulo['tipo_titulo'] == 'LTN':
      calc = LTN(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )

    elif titulo['tipo_titulo'] == 'NTN-B':
      calc = NTNB(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )

    elif titulo['tipo_titulo'] == 'NTN-F':
      calc = NTNF(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )

    elif titulo['tipo_titulo'] == 'LFT':
      calc = LFT(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )
  else:
    print('privado')
    
  titulo['pu_calc']=calc.pu
  titulo['pm_calc']=calc.pm
  titulo['duration_calc']=calc.duration

In [ ]:
t =[x for x in clean_mercado if x['tipo_titulo'] in ['NTN-F', 'NTN-B', 'LFT', 'LTN']]
print('Tipo, Data Venc, Real, Calc, Error')
for x in t: print(f"{x['tipo_titulo']}, {x['data_referencia']}, {x['pu']}, {x['pu_calc']}, {utils.check_error(x['pu_calc'], x['pu'])}")